In [ ]:
from xgboost import XGBRegressor

from optuna.visualization import plot_optimization_history

from code.wrapper import utils

import optuna

import pandas as pd

In [ ]:
from sklearn.preprocessing import RobustScaler, Normalizer

# https://stackoverflow.com/a/23835410

excel_sheet = pd.read_excel("../../Data/New/unfiltered_data.xlsx", sheet_name=["full_train", "full_test",
                                                                            "ionizable_train", "ionizable_test",
                                                                            "neutral_train", "neutral_test"])

full_train: pd.DataFrame = excel_sheet["full_train"]
full_test: pd.DataFrame = excel_sheet["full_test"]

neutral_train: pd.DataFrame = excel_sheet["neutral_train"]
neutral_test: pd.DataFrame = excel_sheet["neutral_test"]

ionizable_train: pd.DataFrame = excel_sheet["ionizable_train"]
ionizable_test: pd.DataFrame = excel_sheet["ionizable_test"]

Scaler = RobustScaler()
Norm = Normalizer()
# TRAIN
X_full_train = full_train.loc[:, full_train.columns != "Log_MP_RATIO"]
y_full_train = full_train["Log_MP_RATIO"]

X_neutral_train = neutral_train.loc[:, neutral_train.columns != "Log_MP_RATIO"]
y_neutral_train = neutral_train["Log_MP_RATIO"]

X_ionizable_train = ionizable_train.loc[:, ionizable_train.columns != "Log_MP_RATIO"]
y_ionizable_train = ionizable_train["Log_MP_RATIO"]
# Scaler.fit(X_full_train)
# X_full_train = pd.DataFrame(Scaler.transform(X_full_train), columns = X_full_train.columns)
#
# Norm.fit(X_full_train)
# X_full_train = pd.DataFrame(Norm.transform(X_full_train), columns=X_full_train.columns)

# TEST
X_full_test = full_test.loc[:, full_test.columns != "Log_MP_RATIO"]
y_full_test = full_test["Log_MP_RATIO"]

X_neutral_test = neutral_test.loc[:, neutral_test.columns != "Log_MP_RATIO"]
y_neutral_test = neutral_test["Log_MP_RATIO"]

X_ionizable_test = ionizable_test.loc[:, ionizable_test.columns != "Log_MP_RATIO"]
y_ionizable_test = ionizable_test["Log_MP_RATIO"]
#
# Scaler.fit(X_full_test)
# X_full_test = pd.DataFrame(Scaler.transform(X_full_test), columns = X_full_test.columns)
#
# Norm.fit(X_full_test)
# X_full_test = pd.DataFrame(Norm.transform(X_full_test), columns=X_full_test.columns)

# Full

In [ ]:
test_utils = utils.Utils(full_train)
test_utils.create_cv_folds(display=True)
test_utils.display_score(XGBRegressor(max_iter=100000, random_state=0), X_full_train, y_full_train, X_full_test,
                         y_full_test)

In [ ]:
def objective(trial):
    param = {
        'max_depth': trial.suggest_int('max_depth', 1, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0.01, 1.0),
        'subsample': trial.suggest_float('subsample', 0.01, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.01, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.01, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.01, 1.0),
    }

    clf = XGBRegressor(random_state=0, **param)
    # n_ft = trial.suggest_int('n_ft', 1, 10, log=True)
    # clf = RFE(Ridge(max_iter=100000, alpha=alpha, solver=solver), n_features_to_select=n_ft)

    estimator = utils.Utils(full_train)
    return estimator.cross_value_score(clf)


study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=1000, n_jobs=-1, show_progress_bar=True)
trial = study.best_trial
print(trial.value, trial.params)

In [ ]:
test_utils.display_score(XGBRegressor(**study.best_params, random_state=0), X_full_train, y_full_train, X_full_test,
                         y_full_test)
display(plot_optimization_history(study))

rr = XGBRegressor(**study.best_params, random_state=0).fit(X_full_train, y_full_train)

test_utils.display_graph(rr, X_full_train, X_full_test, y_full_train, y_full_test)

# Ionizable

In [ ]:
test_utils = utils.Utils(ionizable_train)
test_utils.create_cv_folds(display=True)
test_utils.display_score(XGBRegressor(max_iter=100000, random_state=0), X_ionizable_train, y_ionizable_train,
                         X_ionizable_test, y_ionizable_test)

In [ ]:
def objective(trial):
    param = {
        'max_depth': trial.suggest_int('max_depth', 1, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0.01, 1.0),
        'subsample': trial.suggest_float('subsample', 0.01, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.01, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.01, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.01, 1.0),
    }

    clf = XGBRegressor(random_state=0, **param)

    estimator = utils.Utils(ionizable_train)
    return estimator.cross_value_score(clf)


study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=1000, n_jobs=-1, show_progress_bar=True)
trial = study.best_trial
print(trial.value, trial.params)

In [ ]:
test_utils.display_score(XGBRegressor(**study.best_params, random_state=0), X_ionizable_train, y_ionizable_train,
                         X_ionizable_test, y_ionizable_test)
display(plot_optimization_history(study))

rr = XGBRegressor(**study.best_params, random_state=0).fit(X_ionizable_train, y_ionizable_train)

test_utils.display_graph(rr, X_ionizable_train, X_ionizable_test, y_ionizable_train, y_ionizable_test)

# Neutral

In [ ]:
test_utils = utils.Utils(neutral_train)
test_utils.create_cv_folds(display=True)
test_utils.display_score(XGBRegressor(max_iter=100000, random_state=0), X_neutral_train, y_neutral_train,
                         X_neutral_test, y_neutral_test)

In [ ]:
def objective(trial):
    param = {
        'max_depth': trial.suggest_int('max_depth', 1, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0.01, 1.0),
        'subsample': trial.suggest_float('subsample', 0.01, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.01, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.01, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.01, 1.0),
    }

    clf = XGBRegressor(random_state=0, **param)
    clf.fit(X_neutral_train, y_neutral_train)

    estimator = utils.Utils(neutral_train)
    return estimator.cross_value_score(clf)
    # score = cross_val_score(clf, X_full_train, y_full_train, scoring="r2", cv=3, n_jobs=-1).mean()
    # return score


study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=1000, n_jobs=-1, show_progress_bar=True)
trial = study.best_trial
print(trial.value, trial.params)

In [ ]:
test_utils.display_score(XGBRegressor(**study.best_params, random_state=0), X_neutral_train, y_neutral_train,
                         X_neutral_test, y_neutral_test)
display(plot_optimization_history(study))

rr = XGBRegressor(**study.best_params, random_state=0).fit(X_neutral_train, y_neutral_train)

test_utils.display_graph(rr, X_neutral_train, X_neutral_test, y_neutral_train, y_neutral_test)